# Applying Models after Feature Selection

**Importing Libraries and different datasets**

In [114]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import *
import zipfile
from zipfile import ZipFile
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [115]:
df_knn_imputed_uniform_3 = pd.read_csv('df_knn_imputed_uniform_3.csv')
df_knn_imputed_uniform_5 = pd.read_csv('df_knn_imputed_uniform_5.csv')
df_knn_imputed_uniform_7 = pd.read_csv('df_knn_imputed_uniform_7.csv')
df_knn_imputed_uniform_9 = pd.read_csv('df_knn_imputed_uniform_9.csv')
df_knn_imputed_uniform_11 = pd.read_csv('df_knn_imputed_uniform_11.csv')
df_mean_imputed = pd.read_csv('df_mean_imputed.csv')

In [116]:
df_knn_imputed_uniform_3.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'bgr', 'bu', 'sc', 'sod',
       'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'appet', 'pe',
       'class'],
      dtype='object')

# Applying all the models(LOG, RF, KNN, SVC, NB, MLP and combined model of LOG & RF).

In [117]:
dfs = [df_knn_imputed_uniform_3,df_knn_imputed_uniform_5,
        df_knn_imputed_uniform_7,df_knn_imputed_uniform_9,
        df_knn_imputed_uniform_11,df_mean_imputed]
    
dfs_name = ["knn_imputed_3", "knn_imputed_5",
                "knn_imputed_7","knn_imputed_9",
                "knn_imputed_11","mean_imputed"]

# **Logistic regression Model**

In [32]:
lr_classifier = LogisticRegression()
def log():
    logistic = {}

    for i, df in enumerate(dfs):
        x = df.iloc[:, :-1]
        y = df['class']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, random_state=0)
        
        ss = StandardScaler()
        x_train = ss.fit_transform(x_train)
        x_test = ss.transform(x_test)
        lr_classifier.fit(x_train,y_train)
        
        test_pred = lr_classifier.predict(x_test)
        train_pred = lr_classifier.predict(x_train).round(7)
       
        # print(test_pred)
        train_acc = accuracy_score(y_train, train_pred).round(7)
        test_acc = accuracy_score(y_test, test_pred).round(7)
        
        tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
#         print(classification_report(y_test, test_pred))
        logistic[dfs_name[i]] = [train_acc, test_acc, tn, fp, fn, tp]
        
    return logistic

In [33]:
pd.DataFrame.from_dict(data = log(),orient='index', columns = ['train_acc', 'test_acc', 'true_neg', 'false_pos', 'false_neg', 'true_pos'])

,train_acc,test_acc,true_neg,false_pos,false_neg,true_pos
knn_imputed_3,0.996667,0.98,36,2,0,62
knn_imputed_5,0.996667,0.97,35,3,0,62
knn_imputed_7,1.000000,0.98,36,2,0,62
knn_imputed_9,1.000000,0.97,35,3,0,62
knn_imputed_11,0.996667,0.97,35,3,0,62
mean_imputed,1.000000,0.99,38,0,1,61


**For Logistic Regression, we have false_pos + false_neg is 3,1,2,1,1,0 for respective imputed datasets.**

# **random Forest Model**

In [120]:
rf_classifier = RandomForestClassifier(n_estimators = 500)
def rf():
    rand_forest = {}

    for i, df in enumerate(dfs):
        x = df.iloc[:, :-1]
        y = df['class']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, random_state=42)
        
        ss = StandardScaler()
        x_train = ss.fit_transform(x_train)
        x_test = ss.transform(x_test)
        rf_classifier.fit(x_train,y_train)
        
        test_pred = rf_classifier.predict(x_test)
        train_pred = rf_classifier.predict(x_train)
       
        # print(test_pred)
        train_acc = accuracy_score(y_train, train_pred)
        test_acc = accuracy_score(y_test, test_pred)
        
        tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
#         print(classification_report(y_test, test_pred))
        rand_forest[dfs_name[i]] = [train_acc, test_acc, tn, fp, fn, tp]
        
    return rand_forest

In [121]:
pd.DataFrame.from_dict(data = rf(),orient='index', columns = ['train_acc', 'test_acc', 'true_neg', 'false_pos', 'false_neg', 'true_pos'])

,train_acc,test_acc,true_neg,false_pos,false_neg,true_pos
knn_imputed_3,1.0,0.99,34,1,0,65
knn_imputed_5,1.0,0.98,33,2,0,65
knn_imputed_7,1.0,0.98,33,2,0,65
knn_imputed_9,1.0,0.98,33,2,0,65
knn_imputed_11,1.0,0.98,33,2,0,65
mean_imputed,1.0,1.00,35,0,0,65


# **KNN, SVC, Naive Bayes and MLP models**

In [40]:
def models(x):
    
    if(x==0):
        classifier = KNeighborsClassifier()
    elif(x==1):
        classifier = SVC(kernel='rbf')
    elif(x==2):
        classifier = GaussianNB()
    else:
        classifier = MLPClassifier(hidden_layer_sizes=(13,13,13), max_iter=500)
        
        
    res = {}

    for i, df in enumerate(dfs):
        x = df.iloc[:, :-1]
        y = df['class']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, random_state=0)
        
        ss = StandardScaler()
        x_train = ss.fit_transform(x_train)
        x_test = ss.transform(x_test)
        classifier.fit(x_train,y_train)
        
        test_pred = classifier.predict(x_test)
        train_pred = classifier.predict(x_train)
       
        # print(test_pred)
        train_acc = accuracy_score(y_train, train_pred)
        test_acc = accuracy_score(y_test, test_pred)
        
        tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
#         print(classification_report(y_test, test_pred))
        res[dfs_name[i]] = [train_acc, test_acc, tn, fp, fn, tp]
        
    return res

In [41]:
for i in range(4):
    result = models(i)
    res = pd.DataFrame.from_dict(data = result, orient='index', columns = ['train_acc', 'test_acc', 'true_neg', 'false_pos', 'false_neg', 'true_pos'])
    if(i==0):
        print("\033[1m",i+1,".PERFORMANCE INDICATORS for KNN \n")
    elif(i==1):
        print(i+1,"\033[1m"+".PERFORMANCE INDICATORS for SVC \n")
    elif(i==2):
        print(i+1,"\033[1m"+".PERFORMANCE INDICATORS for Naive Bayes \n")
    else:
        print(i+1,"\033[1m"+".PERFORMANCE INDICATORS for MultiLeyer-perceptron \n")
#     result = models(i)
    print(res,'\n')

 1 .PERFORMANCE INDICATORS for KNN 

                train_acc  test_acc  true_neg  false_pos  false_neg  true_pos
knn_imputed_3    0.976667      0.97        37          1          2        60
knn_imputed_5    0.993333      0.95        37          1          4        58
knn_imputed_7    0.983333      0.98        38          0          2        60
knn_imputed_9    0.986667      0.99        38          0          1        61
knn_imputed_11   0.996667      0.98        38          0          2        60
mean_imputed     0.986667      0.99        38          0          1        61 

2 .PERFORMANCE INDICATORS for SVC 

                train_acc  test_acc  true_neg  false_pos  false_neg  true_pos
knn_imputed_3    0.993333      0.98        36          2          0        62
knn_imputed_5    0.996667      0.98        36          2          0        62
knn_imputed_7    1.000000      0.98        36          2          0        62
knn_imputed_9    0.996667      0.97        35          3          0

# Combined Model of Logistic Regression & Random Forest 

In [104]:
from sklearn.ensemble import VotingClassifier
def combined():

    estimators=[ ('lr', lr_classifier), ('rf', rf_classifier)]

    majority_voting = VotingClassifier(estimators, voting='hard')

    sets = {}

        #dfs: array of imputed df
    dfs = [df_knn_imputed_uniform_3,df_knn_imputed_uniform_5,
            df_knn_imputed_uniform_7,df_knn_imputed_uniform_9,
            df_knn_imputed_uniform_11,df_mean_imputed]

    dfs_name = ["knn_imputed_3", "knn_imputed_5",
                    "knn_imputed_7","knn_imputed_9",
                    "knn_imputed_11","mean_imputed"]

    for i, df in enumerate(dfs):
        x = df.iloc[:, :-1]
        y = df['class']

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25,random_state = 42)

        ss = StandardScaler()
        x_train = ss.fit_transform(x_train)
        x_test = ss.transform(x_test)
        majority_voting.fit(x_train,y_train)

        test_pred = majority_voting.predict(x_test)
        train_pred = majority_voting.predict(x_train)

        # print(test_pred)
        train_acc = accuracy_score(y_train, train_pred)*100
        test_acc = accuracy_score(y_test, test_pred)*100

        tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
#         print(classification_report(y_test, test_pred))
        sets[dfs_name[i]] = [train_acc, test_acc, tn, fp, fn, tp]
    return sets

In [105]:
pd.DataFrame.from_dict(data = combined(), orient='index', columns = ['train_acc', 'test_acc', 'true_neg', 'false_pos', 'false_neg', 'true_pos'])

,train_acc,test_acc,true_neg,false_pos,false_neg,true_pos
knn_imputed_3,99.666667,100.0,35,0,0,65
knn_imputed_5,99.666667,98.0,33,2,0,65
knn_imputed_7,100.000000,99.0,34,1,0,65
knn_imputed_9,100.000000,98.0,33,2,0,65
knn_imputed_11,99.666667,98.0,33,2,0,65
mean_imputed,99.666667,100.0,35,0,0,65


**CONCLUSION**

Here we can see that after applying **combined model of Logistic regression and Random Forest**, we get the accuracy rate more than the individual model of Logistic and Random Forest.
So, this combined model is the good enough to apply on CKD prediction 